In [27]:
import pandas as pd
import altair as alt
from vega_datasets import data



#cargar datos
data = pd.read_csv('BASE_DE_DATOS_DE_CONCIERTS_FINAL.csv', sep=';')

#limpiar y procesar los datos
#eliminar columnas que no van a ser utilizadas
columnas_relevantes = ['GRUPO','AÑO','MERCADO','ENTRADA MÁS BARATA','ENTRADA MÁS CARA']
df = data[columnas_relevantes].dropna()



# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()



df["entrada más barata"] = pd.to_numeric(df["entrada más barata"], errors="coerce")
df["entrada más cara"] = pd.to_numeric(df["entrada más cara"], errors="coerce")



df["precio_promedio"] = (df["entrada más barata"] + df["entrada más cara"]) / 2


df = df.dropna(subset=["año", "precio_promedio"])


df_grouped = df.groupby("año").agg(
    precio=('precio_promedio', 'mean')
).reset_index()


df_grouped["año"] = pd.to_datetime(df_grouped["año"], format="%Y")


line = alt.Chart(df_grouped).mark_line(point=True).encode(
    x=alt.X("año:T", title="Año"),
    y=alt.Y("precio:Q", title="Precio promedio de entrada (USD)")
).properties(
    title="Evolución del precio promedio de entradas a conciertos de K-pop por año",
    width=600
)


text = alt.Chart(df_grouped).mark_text(align='left', dx=5, dy=-5).encode(
    x="año:T",
    y="precio:Q",
    text=alt.Text("precio:Q", format=".0f")
)





df = df[(df["entrada más barata"] <= 2000) & (df["entrada más cara"] <= 2000)]


mercados = [
    "corea del sur", "japón", "china", "sudeste asiatico",
    "américa latina", "américa del norte", "europa", "méxico", "brasil", "estados unidos"
]
df = df[df["mercado"].str.lower().isin(mercados)]
df["mercado"] = df["mercado"].str.title()


df_grouped = df.groupby("mercado").agg({
    "entrada más barata": "min",
    "entrada más cara": "max"
}).reset_index()


bar = alt.Chart(df_grouped).mark_bar(cornerRadius=8, height=14).encode(
    x=alt.X('entrada más barata:Q', scale=alt.Scale(domain=[0, 2000]), title='Precio más barato (USD)'),
    x2='entrada más cara:Q',
    y=alt.Y('mercado:N', sort='-x', title='Mercado'),
    color=alt.Color('mercado:N', legend=None)
)

text_min = alt.Chart(df_grouped).mark_text(align='right', dx=-5).encode(
    x='entrada más barata:Q',
    y=alt.Y('mercado:N', sort='-x'),
    text='entrada más barata:Q'
)

text_max = alt.Chart(df_grouped).mark_text(align='left', dx=5).encode(
    x='entrada más cara:Q',
    y=alt.Y('mercado:N', sort='-x'),
    text='entrada más cara:Q'
)


chart = (bar + text_min + text_max).properties(
    title=alt.TitleParams(
        text='Rango de precios de entradas por mercado',
        subtitle='Precios entre $0 y $2000 USD'
    ),
    width=600,
    height=400
)

chart


line_chart = line + text


range_chart = (bar + text_min + text_max).properties(
    title=alt.TitleParams(
        text='Rango de precios de entradas por mercado',
        subtitle='Precios entre $0 y $2000 USD'
    ),
    width=600,
    height=400
)

# Mostrar ambos gráficos (en Jupyter, Colab, etc.)
line_chart & range_chart


alt.VConcatChart(...)

In [5]:
pip install panda

  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=9be436a0d26257d71f3f782fc79221b59397e5f939f9fd73da4b590ad2e1db22
  Stored in directory: /root/.cache/pip/wheels/df/5c/39/36f8dae25a1e88d6ec4411dec4a143781e64fdff6897758eec
Successfully built panda


In [ ]:
source = data.stocks()

base = alt.Chart(source).encode(
    alt.Color("symbol").legend(None)
).transform_filter(
    "datum.symbol !== 'IBM'"
).properties(
    width=500
)

line = base.mark_line().encode(x="date", y="price")


last_price = base.mark_circle().encode(
    alt.X("last_date['date']:T"),
    alt.Y("last_date['price']:Q")
).transform_aggregate(
    last_date="argmax(date)",
    groupby=["symbol"]
)

company_name = last_price.mark_text(align="left", dx=4).encode(text="symbol")

chart = (line + last_price + company_name).encode(
    x=alt.X().title("date"),
    y=alt.Y().title("price")
)

chart


In [29]:
pip install altair_saver vega_datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.5 MB/s eta 0:00:00


In [36]:
import altair_saver
# Mostrar ambos gráficos (en Jupyter, Colab, etc.)
line_chart & range_chart

altair_saver.save(line_chart, 'line_chart.html')
altair_saver.save(range_chart, 'range_chart.html')

altair_saver.save(line_chart, 'line_chart.png')
altair_saver.save(range_chart, 'range_chart.png')

NoMatchingVersions: No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer